### trainデータセット作成

In [ ]:
mount='/home/hattori/dataset' # マウントポイント（絶対パスで）
exp_name='0730' # 実験名
n=0.3 #検証データ割合

exp_dir='./'+exp_name+'/datasets'

In [ ]:
# ATR503    (503)   (path,char,kana(atr),phone(atr))  
atr503_csv=mount+'/atr503/atr503.csv'

# CSJ
## eval1    (1272)  (path,char)
csj_eval1_csv=mount+'/CSJ_dataset/label/eval1/eval1.csv'
## eval2    (1292)  (path,char)
csj_eval2_csv=mount+'/CSJ_dataset/label/eval2/eval2.csv'
## eval3    (1382)  (path,char)
csj_eval3_csv=mount+'/CSJ_dataset/label/eval3/eval3.csv'
## train_dev (4000) (path,char)
csj_train_dev_csv=mount+'/CSJ_dataset/label/train_dev/csj_train_dev.csv'
## train_nodup (403095) (path,char)
csj_train_nodup_csv=mount+'/CSJ_dataset/label/train_nodup/csj_train_nodup.csv'

# CSS10 (6841)  (path,char)
css10_csv=mount+'/CSS10/ja/CSS10_meian.csv'

# FW07 (134400) (path, noise, snr, char, phone)
fw07_csv=mount+'/FW07/fw07_jdrt_5noise/fw07_jdrt5noise.csv'

# JDRT
## noisy05_sp7_5N   (141120)    (path, word_no, speaker, sex, noise, snr, char, kana, katakana, roman, phone, speed)
noisy05_sp7_5N_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_noisy05_sp7_5N.csv'
## clean05_sp7   (6720)    (path, word_no, speaker, sex, noise, snr, char, kana, katakana, roman, phone, speed)
clean05_sp7_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_clean05_sp7.csv'
## clean   (960)    (path, word_no, speaker, sex, noise, snr, char, kana, katakana, roman, phone)
clean_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_clean.csv'
## clean05   (960)    (path, word_no, speaker, sex, noise, snr, char, kana, katakana, roman, phone)
clean05_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_clean05.csv'
## noisy05_5N   (20160)    (path, word_no, speaker, sex, noise, snr, char, kana, katakana, roman, phone)
noisy05_5N_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_noisy05_5N.csv'

# JSSS concat (4877) (path, char)
jsss_csv=mount+'/jsss_ver1/jsss_concat.csv'

# JSUT concat (7696) (path, char)
jsut_csv=mount+'/jsut_ver1.1/jsut_concat.csv'

# JVS concat (14997) (path, char)
jvs_csv=mount+'/jvs_ver1/jvs_concat.csv'

readme=mount+'/readme.txt'
#!cat $readme

In [ ]:
import numpy as np
import pandas as pd
import warnings
import datasets
warnings.filterwarnings('ignore')

In [ ]:
#train_dataset
train = datasets.load_dataset("csv", data_files={"train":[atr503_csv,css10_csv,jsut_csv,jsss_csv,jvs_csv]},usecols=['path','char'])

#validation dataset
#val = datasets.load_dataset("csv", data_files={'test':[csj_train_dev_csv]})
sets=[train] #add'val'

In [ ]:
#show
from datasets import ClassLabel
import random
from IPython.display import display, HTML

def show_random(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
#show_random({dataset}['test'],2))

In [ ]:
#rename path
def renamepath(batch):
    batch['path']=mount+str(batch['path'])[1:]
    return batch

In [ ]:
# renamepath
for s in sets:s['train']=s['train'].map(renamepath)

In [ ]:
import re
CHARS_TO_IGNORE = [",", "?", "¿", ".", "!", "¡", ";", "；", ":", '""', "%", '"', "�", "ʿ", "·", "჻", "~", "՞",
                   "؟", "،", "।", "॥", "«", "»", "„", "“", "”", "「", "」", "‘", "’", "《", "》", "(", ")", "[", "]",
                   "{", "}", "=", "`", "_", "+", "<", ">", "…", "–", "°", "´", "ʾ", "‹", "›", "©", "®", "—", "→", "。",
                   "、", "﹂", "﹁", "‧", "～", "﹏", "，", "｛", "｝", "（", "）", "［", "］", "【", "】", "‥", "〽",
                   "『", "』", "〝", "〟", "⟨", "⟩", "〜", "：", "！", "？", "♪", "؛", "/", "\\", "º", "−", "^", "'", "ʻ", "ˆ","「","」","　","｡"]
chars_to_ignore_regex = f"[{re.escape(''.join(CHARS_TO_IGNORE))}]"

def remove_special_characters(batch):
    #s=str(batch['char'])
    batch["conv_char"] = re.sub(chars_to_ignore_regex, ' ', str(batch["char"])).upper()
    return batch

In [ ]:
# remove_special_char
for s in sets: s['train']=s['train'].map(remove_special_characters)

In [ ]:
import MeCab
import unidic
import romkan
#!python -m unidic download #最新辞書
mecab = MeCab.Tagger()

In [ ]:
# char2kana（元々カナは出力されないので場合分け）
def convert_char_to_kana(batch):
    s = mecab.parse(batch["conv_char"])
    kana = ""
    for line in s.split("\n"):
      if line.find("\t")<=0: continue
      columns = line.split(',')
      if len(columns) < 10:
        kana += line.split('\t')[0]
      else:
        kana += columns[9]
    batch["kana"] = kana
    return batch

In [ ]:
# conv_char2kana
for s in sets: s['train']=s['train'].map(convert_char_to_kana)

In [ ]:
# kana2roman
def convert_kana_to_roman(batch):
    s = mecab.parse(batch["kana"])
    kana = ""
    for line in s.split("\n"):
      if line.find("\t")<=0: continue
      columns = line.split(',')
      if len(columns) < 10:
        kana += line.split('\t')[0]
      else:
        kana += columns[9]
    roman = romkan.to_roma(kana)
    batch["roman"] = roman
    return batch

In [ ]:
# conv_kana2roman
for s in sets: s['train']=s['train'].map(convert_kana_to_roman)

In [ ]:
# kana2phoneme (2通り。Mecab等を仲介するか、直接か)
# openjtalk options: https://note.com/npaka/n/n6a5307cf8fe1

import pyopenjtalk
def convert_char2phone_openjtalk(batch):
    s = batch["conv_char"]
    phones = pyopenjtalk.g2p(s, kana=False)
    batch["phone"] = phones
    return batch

def convert_kana2phone(batch):
    s = batch["kana"]
    phones = pyopenjtalk.g2p(s, kana=False)
    batch["phone"] = phones
    return batch

In [ ]:
# conv_kana2phoneme
for s in sets: s['train']=s['train'].map(convert_char2phone_openjtalk)

In [ ]:
#被り計測 
def extract_all_chars_kana(batch):
  all_text = " ".join(batch["phone"])
  vocab = list(set(all_text))
  return {"vocab": [vocab]}

vocab_train_kana = train.map(extract_all_chars_kana)
a=vocab_train_kana['train']['vocab']

import itertools
b=list(itertools.chain.from_iterable(a))
c=list(itertools.chain.from_iterable(b))
d=set(c)
print(len(d))
d

In [ ]:
train=train['train'].train_test_split(test_size=n,shuffle=True)

In [ ]:
#save datasets
train['train'].to_csv(exp_dir+'/train_phone.csv',columns=['path','phone'])
train['test'].to_csv(exp_dir+'/val_phone.csv',columns=['path','phone'])
#val['test'].to_csv(exp_dir+'/val_phone.csv',columns=['path','phone'])